<a href="https://colab.research.google.com/github/Saranyanv/data-go/blob/master/Misc/sudoku_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import random

def difference(list1):
   list_dif = [i for i in list1 + [1,2,3,4,5,6,7,8,9] if (i not in list1 or i not in [1,2,3,4,5,6,7,8,9]) and i!=0]
   return list_dif

def uniq_append(element, list1):
  if element not in list1:
    list1.append(element)

In [0]:
rows, cols = (9, 9) 
#arr = [[random.randint(0,9) for i in range(rows)] for i in range(cols)]

#Difficulty=Easy
arr=[[5,2,4,6,0,0,0,0,1],[0,1,0,8,4,3,0,6,5],[3,0,0,0,1,0,0,0,0],[7,5,2,0,0,0,1,0,0],[0,0,0,1,0,7,0,0,0],[0,0,6,0,0,0,7,5,8],[0,0,0,0,9,0,0,0,2],[4,7,0,2,8,6,0,1,0],[2,0,0,0,0,5,6,7,9]]

#Difficulty=Medium
#arr=[[0,0,9,0,8,2,0,0,5],[0,0,0,9,0,0,1,0,0],[0,5,3,1,0,0,9,8,7],[3,0,7,0,9,1,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,6,2,0,3,0,9],[6,9,5,0,0,8,4,3,0],[0,0,4,0,0,9,0,0,0],[1,0,0,4,3,0,5,0,0]]

In [53]:
#old
arr

[[5, 2, 4, 6, 0, 0, 0, 0, 1],
 [0, 1, 0, 8, 4, 3, 0, 6, 5],
 [3, 0, 0, 0, 1, 0, 0, 0, 0],
 [7, 5, 2, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 7, 0, 0, 0],
 [0, 0, 6, 0, 0, 0, 7, 5, 8],
 [0, 0, 0, 0, 9, 0, 0, 0, 2],
 [4, 7, 0, 2, 8, 6, 0, 1, 0],
 [2, 0, 0, 0, 0, 5, 6, 7, 9]]

In [0]:
def sudoku(arg_list):
  cntr=0
  while True:
    print ("Pass Nbr #",cntr)
    for i in range(rows):
      for j in range(cols):
        if arg_list[i][j]==0:
          #get numbers in that row
          tmp=[i for i in arg_list[i]]
          #get numbers in that column
          for n in range(rows):
            uniq_append(arg_list[n][j],tmp)
          #get numbers in that square
          for i1 in range(i-np.mod(i,3), i+np.mod(8-i,3)+1):
            for j1 in range(j-np.mod(j,3), j+np.mod(8-j,3)+1):
              uniq_append(arg_list[i1][j1],tmp)
          if len(difference(tmp))==1:
            arg_list[i][j]=sum(int(x) for x in difference(tmp))
    not_filled=len([i for i in arg_list if 0 in i])
    if not_filled>0:
      print ("  Number of unfilled positions is ",not_filled)
    else:
      print ("Sudoku Challenge complete in",cntr,"passes!")
      print (" ")
      print(np.matrix(arg_list))
      return False
    cntr+=1

In [55]:
sudoku(arr)

Pass Nbr # 0
  Number of unfilled positions is  8
Pass Nbr # 1
  Number of unfilled positions is  7
Pass Nbr # 2
  Number of unfilled positions is  6
Pass Nbr # 3
  Number of unfilled positions is  5
Pass Nbr # 4
  Number of unfilled positions is  1
Pass Nbr # 5
Sudoku Challenge complete in 5 passes!
 
[[5 2 4 6 7 9 3 8 1]
 [9 1 7 8 4 3 2 6 5]
 [3 6 8 5 1 2 4 9 7]
 [7 5 2 9 6 8 1 3 4]
 [8 4 3 1 5 7 9 2 6]
 [1 9 6 3 2 4 7 5 8]
 [6 3 5 7 9 1 8 4 2]
 [4 7 9 2 8 6 5 1 3]
 [2 8 1 4 3 5 6 7 9]]


False

In [56]:
#new
arr

[[5, 2, 4, 6, 7, 9, 3, 8, 1],
 [9, 1, 7, 8, 4, 3, 2, 6, 5],
 [3, 6, 8, 5, 1, 2, 4, 9, 7],
 [7, 5, 2, 9, 6, 8, 1, 3, 4],
 [8, 4, 3, 1, 5, 7, 9, 2, 6],
 [1, 9, 6, 3, 2, 4, 7, 5, 8],
 [6, 3, 5, 7, 9, 1, 8, 4, 2],
 [4, 7, 9, 2, 8, 6, 5, 1, 3],
 [2, 8, 1, 4, 3, 5, 6, 7, 9]]